### Install project

In [1]:
!git clone https://github.com/kornelro/cv_project.git

Cloning into 'cv_project'...
remote: Enumerating objects: 488, done.
remote: Counting objects: 100% (488/488), done.
remote: Compressing objects: 100% (343/343), done.
remote: Total 766 (delta 181), reused 288 (delta 79), pack-reused 278
Receiving objects: 100% (766/766), 103.91 MiB | 37.99 MiB/s, done.
Resolving deltas: 100% (327/327), done.


In [2]:
!pip install cv_project/.

Processing ./cv_project
  Created wheel for src: filename=src-0.1.0-cp36-none-any.whl size=16556 sha256=9fe845b8fce6f6b1f6ee927d8885636d2a224ea76e4d9aeee11dcf1a175368a9
  Stored in directory: /tmp/pip-ephem-wheel-cache-9tkdrkpz/wheels/fc/1b/40/72379bece2bafba26136079aeac87421c9221abafeb9c66746
Successfully built src


### Mount drive

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
DRIVE_PATH = '/content/drive/Shareddrives/CV-PROJECT/'

### Imports

In [5]:
import pickle
import numpy as np
import tensorflow.keras as K
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from src.data.make_dataset import make_dataset
from src.data.image_loaders import OrtoFixedSizeImageLoader
from src.data.dataset_loaders import OrtoDatasetLoader
from src.features.pipelines import RawImageToImage
from src.features.processors import NormImage
from src.models.classifiers import ResnetModelWrapper
from src.models.detectors import SliderProbDetector
from src.data.sliders import SlidingWindow
from src.evaluation import validate_model, precision, recall, f1_score

In [6]:
tf.random.set_seed(1234)

# Orto
# ResNet50 1 epoch
# Pipeline: Norm

### Load dataset

In [7]:
DATA_PATH = DRIVE_PATH+'data/processed/aerial-cars-dataset/resnet/data_resnet.pickle'

In [8]:
with open(DATA_PATH, 'rb') as f:
    data = pickle.load(f)

### Train ResNet50

In [9]:
TEST_SIZE = 0.1
BATCH_SIZE = 100
EPOCHS = 1

In [10]:
X = np.array(list(map(lambda x: x[1], data)))
y = np.array(list(map(lambda x: x[2], data)))
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=TEST_SIZE,
    random_state=42
  )


input_t = K.Input(shape=(X[0].shape))
res_model = K.applications.ResNet50V2(
    include_top=False,
    weights='imagenet',
    input_tensor=input_t
)

for layer in res_model.layers:
    layer.trainable = False

model = K.models.Sequential()
model.add(res_model)
model.add(K.layers.Flatten())
model.add(K.layers.Dense(2, activation='softmax'))


model.compile(
    optimizer=K.optimizers.Adam(0.001),
    loss=K.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

history = model.fit(X_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS, 
            validation_data=(X_test, y_test)
)

43/43 [==============================] - 13s 88ms/step - loss: 0.4740 - accuracy: 0.8926 - val_loss: 0.1319 - val_accuracy: 0.9746


### Validation

In [11]:
del data

In [12]:
BND_BOX_SIZE = (80, 80)
STEP_SIZE = 20

dataset_loader = OrtoDatasetLoader(
    image_loader=OrtoFixedSizeImageLoader(
        bnd_box_size=BND_BOX_SIZE
    )
)

process_pipeline=RawImageToImage(
    processors=[
        NormImage()
    ]
)

sliding_window = SlidingWindow(
    step_size=STEP_SIZE,
    window_size=BND_BOX_SIZE
)

treshold = 0.9; nms_overlap=0.4

In [13]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t9nms4_1e',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

413


 40%|████      | 2/5 [00:29<00:46, 15.65s/it]

95
332


 60%|██████    | 3/5 [00:42<00:29, 14.88s/it]

319


 80%|████████  | 4/5 [00:55<00:14, 14.33s/it]

513


100%|██████████| 5/5 [01:09<00:00, 13.85s/it]



Precision: 0.475
Recall: 0.598
F1Score: 0.53


treshold = 0.95; nms_overlap=0.4

In [14]:
del processed_images

In [15]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.95,
    nms_overlap=0.4
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t95nms4_1e',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

381


 40%|████      | 2/5 [00:26<00:40, 13.34s/it]

88
312


 60%|██████    | 3/5 [00:40<00:26, 13.37s/it]

295


 80%|████████  | 4/5 [00:53<00:13, 13.39s/it]

478


100%|██████████| 5/5 [01:07<00:00, 13.46s/it]



Precision: 0.488
Recall: 0.567
F1Score: 0.525


treshold = 0.98; nms_overlap=0.4

In [16]:
del processed_images

In [17]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.3
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t98nms4_1e',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

413


 40%|████      | 2/5 [00:27<00:40, 13.52s/it]

95
332


 80%|████████  | 4/5 [00:54<00:13, 13.54s/it]

319
513


100%|██████████| 5/5 [01:07<00:00, 13.55s/it]



Precision: 0.521
Recall: 0.505
F1Score: 0.513


treshold = 0.9; nms_overlap=0.5

In [18]:
del processed_images

In [19]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.5
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t9nms5_1e',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

  0%|          | 0/5 [00:00<?, ?it/s]

413


 40%|████      | 2/5 [00:27<00:40, 13.53s/it]

95


 60%|██████    | 3/5 [00:40<00:27, 13.57s/it]

332


 80%|████████  | 4/5 [00:54<00:13, 13.51s/it]

319
513


100%|██████████| 5/5 [01:07<00:00, 13.51s/it]



Precision: 0.475
Recall: 0.598
F1Score: 0.53


treshold = 0.9; nms_overlap=0.3

In [20]:
del processed_images

In [21]:
slider_detector = SliderProbDetector(
    sliding_window=sliding_window,
    process_pipeline=process_pipeline,
    classifier=ResnetModelWrapper(model),
    treshold=0.9,
    nms_overlap=0.3
)

true_positives, false_positives, false_negatives, processed_images = validate_model(
    dataset_loader=dataset_loader,
    input_folder_filepath=DRIVE_PATH+'data/raw/orto/val',
    output_folder_filepath=DRIVE_PATH+'processed_images/resnet50/orto/norm/t9nms3_1e',
    images_files_types=('png',),
    annotations_files_types=('json',),
    detector=slider_detector,
    workers=0
)


print()
print('Precision:',  round(precision(true_positives, false_positives), 3))
print('Recall:',  round(recall(true_positives, false_negatives), 3))
print('F1Score:', round(f1_score(true_positives, false_positives, false_negatives), 3))

 20%|██        | 1/5 [00:13<00:53, 13.45s/it]

413


 40%|████      | 2/5 [00:27<00:40, 13.48s/it]

95
332


 80%|████████  | 4/5 [00:54<00:13, 13.61s/it]

319
513


100%|██████████| 5/5 [01:08<00:00, 13.63s/it]



Precision: 0.521
Recall: 0.505
F1Score: 0.513
